## 데이터 Load

- 데이터를 입력합니다. 

```SQL
$ psql –U postgres
psql (13.3)
Type "help" for help.

postgres=# \cd step01_basic
postgres=# \i countries.sql
```



## Subquery with WHERE and SELECT
- 사용 데이터: populations, cities, countries

### Question 1
- 먼저 2015년 전체 국가의 평균 수명을 계산하십시오.

```SQL
SELECT AVG(life_expectancy)
FROM populations
WHERE year = 2015;
```
```
       avg        
------------------
 71.6763415481105
(1 row)
```

### Question 2
- 조건 1. 모든 데이터를 조회한다. 
- 조건 2. 2015년 평균 기대수명의 1.15배보다 높도록 설정한다. (life_expectancy > 1.15 * ???)


```SQL
SELECT *
FROM populations
WHERE life_expectancy > 1.15 * (SELECT AVG(life_expectancy) FROM populations WHERE year = 2015) 
	AND year = 2015
LIMIT 5;
```
```
 pop_id | country_code | year | fertility_rate | life_expectancy |     size      
--------+--------------+------+----------------+-----------------+---------------
     21 | AUS          | 2015 |          1.833 |        82.45122 | 2.3789752e+07
    376 | CHE          | 2015 |           1.54 |        83.19756 |   8.28143e+06
    356 | ESP          | 2015 |           1.32 |       83.380486 | 4.6443992e+07
    134 | FRA          | 2015 |           2.01 |        82.67073 | 6.6538392e+07
    170 | HKG          | 2015 |          1.195 |       84.278046 |    7.3057e+06
(5 rows)

```

### Question 3
- 서브 쿼리의 국가 테이블에 있는 capital 필드를 사용합니다.
- cities 테이블에서 name, country code, urban area population 필드를 조회한다.

```SQL
-- 2. 필드 선택
SELECT name, country_code, urbanarea_pop
  -- 3. From cities
  FROM cities
-- 4. name == capital from countries
WHERE name IN
  -- 1. 서브쿼리
  (SELECT capital
   FROM countries)
ORDER BY urbanarea_pop DESC 
LIMIT 5;
```
```
  name   | country_code | urbanarea_pop 
---------+--------------+---------------
 Beijing | CHN          |    2.1516e+07
 Dhaka   | BGD          | 1.4543124e+07
 Tokyo   | JPN          | 1.3513734e+07
 Moscow  | RUS          | 1.2197596e+07
 Cairo   | EGY          |  1.023035e+07
(5 rows)

```

### Question 4. 

- 조건 1. 첫번째 주석을 풀고 실행합니다. 

```SQL
/*
SELECT countries.name AS country, COUNT(*) AS cities_num
  FROM cities
	  INNER JOIN countries
	  ON countries.code = cities.country_code
GROUP BY country
ORDER BY cities_num DESC, country
LIMIT 9;
*/

/* 
SELECT ___ AS ___,
  (SELECT ___
   FROM ___
   WHERE countries.code = cities.country_code) AS cities_num
FROM ___
ORDER BY ___ ___, ___
LIMIT 9;
*/
```
```SQL
SELECT countries.name AS country, COUNT(*) AS cities_num
  FROM cities
	  INNER JOIN countries
	  ON countries.code = cities.country_code
GROUP BY country
ORDER BY cities_num DESC, country
LIMIT 9;
```
```
      country       | cities_num 
--------------------+------------
 China              |         36
 India              |         18
 Japan              |         11
 Brazil             |         10
 Pakistan           |          9
 United States      |          9
 Indonesia          |          7
 Russian Federation |          7
 South Korea        |          7
(9 rows)
```

- 조건 2. GROUP BY 코드를 변환하여 SELECT 내부의 하위 쿼리를 사용하도록 한다. 즉, 첫 번째 쿼리에서 GROUP BY 코드를 사용하여 주어진 결과와 일치하는 결과를 얻으려면 빈칸을 채운다.

- 조건 3. 다시 city_num 내림차순으로 결과를 정렬한 다음 국가 오름차순으로 정렬합니다.

```SQL
SELECT countries.name AS country,
  (SELECT COUNT(*)
   FROM cities
   WHERE countries.code = cities.country_code) AS cities_num
FROM countries
ORDER BY cities_num DESC, country
LIMIT 9;
```
```
      country       | cities_num 
--------------------+------------
 China              |         36
 India              |         18
 Japan              |         11
 Brazil             |         10
 Pakistan           |          9
 United States      |          9
 Indonesia          |          7
 Russian Federation |          7
 South Korea        |          7
(9 rows)
```

## Subquery with FROM
- 사용 데이터: languages, countries, economies

### Question 1. 
- 조건1. 각 국가 코드에 대해 SELECT, FROM 및 GROUP BY를 사용하여 언어 테이블에 나열되는 언어 개수 수를 구한다. 
- 조건2. 집계된 필드는 lang_num으로 표시한다. (Alias)

```SQL
SELECT code, COUNT(*) AS lang_num
  FROM languages
GROUP BY code
LIMIT 5;
```
```
 code | lang_num 
------+----------
 PRY  |        2
 NRU  |        3
 MDG  |        3
 ASM  |        5
 TZA  |        4
(5 rows)

```

### Question 2
- 조건1. 위 쿼리를 FROM 절에 서브쿼리로 사용합니다. 
- 조건2. countries 데이터에서 local_name 필드를 조회합니다. 
- 조건3. 서브쿼리에서 lang_num을 조건2와 같이 조회합니다. 
- 조건4. countries code와 서브쿼리로 사용된 code와 반드시 일치해야 합니다. 
- 조건5. lang_num 내림차순으로 정렬합니다. 

```SQL
SELECT local_name, subquery.lang_num
  FROM countries,
  	(SELECT code, COUNT(*) AS lang_num
  	 FROM languages
  	 GROUP BY code) AS subquery
  WHERE countries.code = subquery.code
ORDER BY lang_num DESC;
```

### Question 3
- 조건1. 아래 쿼리를 실행합니다. 

```SQL
SELECT name, continent, inflation_rate
  FROM countries
  	INNER JOIN economies
    USING (code)
WHERE year = 2015
LIMIT 5;
```
```
         name         |   continent   | inflation_rate 
----------------------+---------------+----------------
 Afghanistan          | Asia          |         -1.549
 Angola               | Africa        |         10.287
 Albania              | Europe        |          1.896
 United Arab Emirates | Asia          |           4.07
 Argentina            | South America |               
(5 rows)

```

- 조건 2. 위 쿼리를 FROM 이내의 서브쿼리를 활용한다. 
- 조건 3. 2015년의 continent 그룹별로 하여 continent, inflation_rate의 최댓값을 조회한다.

```SQL
-- Select the maximum inflation rate as max_inf
SELECT continent, MAX(inflation_rate) AS max_inf
  -- Subquery using FROM (alias as subquery)
  FROM (
      SELECT name, continent, inflation_rate
      FROM countries
      INNER JOIN economies
      USING (code)
      WHERE year = 2015) AS subquery
-- Group by continent
GROUP BY continent;
```
```
   continent   | max_inf 
---------------+---------
 Africa        |  21.858
 Asia          |  39.403
 South America | 121.738
 North America |   7.524
 Europe        |  48.684
 Oceania       |   9.784
(6 rows)

```

- 각 대륙별 inflation_rate가 가장 높은 나라를 추출하는 코드를 작성하도록 합니다. 
- 조건 4. 위 쿼리에서 continent 필드 조회하는 것만 제외합니다. 그리고, 해당 쿼리를 다음 쿼리에서 IN 이하절로 활용합니다. 
    + 문제 SQL은 아래와 같습니다. 
    + Using(code) 대신 ON 코드로 작성합니다. 
    
```
SELECT name, continent, inflation_rate
  -- From countries
  FROM countries
	-- INNER JOIN economies
	INNER JOIN economies
	-- ON
	___
  -- Where year is 2015
  WHERE year = 2015
    -- (alias as subquery)
    ___ ___ ___ (
        SELECT MAX(inflation_rate) AS max_inf
        FROM (
             SELECT name, continent, inflation_rate
             FROM countries
             INNER JOIN economies
             -- Using(code) 대신 ON 쿼리를 작성합니다. 
             ___
             WHERE year = 2015) AS subquery
      -- Group by continent
        GROUP BY continent);
```

- 솔루션은 다음과 같습니다. 

```
-- Select fields
SELECT name, continent, inflation_rate
  -- From countries
  FROM countries
	-- Join to economies
	INNER JOIN economies
	-- Match on code
	ON countries.code = economies.code
  -- Where year is 2015
  WHERE year = 2015
    -- And inflation rate in subquery (alias as subquery)
    AND inflation_rate IN (
        SELECT MAX(inflation_rate) AS max_inf
        FROM (
             SELECT name, continent, inflation_rate
             FROM countries
             INNER JOIN economies
             -- Using(code) 대신 ON 쿼리를 작성합니다. 
             ON countries.code = economies.code
             WHERE year = 2015) AS subquery
      -- Group by continent
        GROUP BY continent);
```
```
   name    |   continent   | inflation_rate 
-----------+---------------+----------------
 Haiti     | North America |          7.524
 Malawi    | Africa        |         21.858
 Nauru     | Oceania       |          9.784
 Ukraine   | Europe        |         48.684
 Venezuela | South America |        121.738
 Yemen     | Asia          |         39.403
(6 rows)

```



### 서브쿼리 Challenge
- 조건 1. economies data에서 country code, inflation rate, unemployment rate를 조회한다. 
- 조건 2. inflation rate 오름차순으로 정렬한다. 
- 조건 3. Alias 사용하지 않는다. 
- 조건 4. countries 테이블내 gov_form 컬럼에서 Constitutional Monarchy 또는 `Republic`이 들어간 국가는 제외한다. 

```SQL
-- Select fields
SELECT code, inflation_rate, unemployment_rate
  -- From economies
  FROM economies
  -- Where year is 2015 and code is not in
  WHERE year = 2015 AND code NOT IN
  	-- Subquery
  	(SELECT code
  	 FROM countries
  	 WHERE (gov_form = 'Constitutional Monarchy' OR gov_form LIKE '%Republic%'))
-- Order by inflation rate
ORDER BY inflation_rate
LIMIT 5;
```
```
 code | inflation_rate | unemployment_rate 
------+----------------+-------------------
 AFG  |         -1.549 |                  
 CHE  |          -1.14 |             3.178
 PRI  |         -0.751 |                12
 ROU  |         -0.596 |             6.812
 BRN  |         -0.423 |               6.9
(5 rows)

```